In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import gensim
from gensim.models.doc2vec import TaggedDocument
from konlpy.tag import Komoran
import cv2
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from PIL import Image
import PIL
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.transforms as transforms
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import keras
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, precision_recall_curve, accuracy_score,roc_auc_score
from imblearn.over_sampling import ADASYN
from collections import Counter

In [6]:
df = pd.read_csv('./train.csv')

In [15]:
# 라벨 인코더 생성
encoder = LabelEncoder()

# X_train데이터를 이용 피팅하고 라벨숫자로 변환한다
encoder.fit(df['cat3'])
#df['cat1'] = encoder.transform(df['cat1'])
df['cat3'] = encoder.transform(df['cat3'])

In [17]:
Y=df['cat1']

In [18]:
Y.value_counts()

4    5614
3    4912
0    2611
5    1719
2    1434
1     696
Name: cat1, dtype: int64

# concat and Vector extraction

In [20]:
df

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,5,자연관광지,120
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,0,육상 레포츠,8
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,3,음식점,118
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,3,음식점,118
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,3,음식점,118
...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,3,음식점,118
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,2,숙박시설,31
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,2,숙박시설,31
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,0,육상 레포츠,73


In [69]:
df['overview'][2677]

'1982. 6. 26 경포도립공원으로 지정면적 9.471㎢, 강원도 고시 제90호 되었으며, 강릉시 중심가에서 북쪽으로 약 6km 떨어져 있는 경포대, 경포호, 경포해수욕장을 중심으로 많은 문화재와 기념물 등이 있는 지역이다. 특히 경포호는 천연기념물인 고니, 청둥오리 등이 도래 하는 곳이기도 하다.\n\n관동 8경 중의 하나인 경포대는 대보름 달맞이 장소로 유명하며, 경포호에서의 낚시는 이 곳을 찾는 이의 여유로움을 돋우기에 충분한 곳이다.현재는 자연보호지역으로 낚시를 금지하고 있다.\n\n동해안 제일의 해수욕장인 맑고 깨끗한 경포해수욕장과 아름다운 경포호, 경포대, 방해정, 호해정, 경호정, 경양사 등이 산재하여 있으며 여름철 해수욕 뿐만 아니라 봄 벚꽃놀이, 연중가능한 낚시, 겨울바다의 낭만 등을 즐길 수 있는 사계절 관광지로 면모를 일신하고 있으며 강원도 특산인 오징어회를 비롯하여 각종 싱싱한 회, 매운탕, 초당두부 등 강릉의 독특한 맛을 찾을수 있다.\n\n* 경포팔경\n 1 녹두일출\n녹두정에서 동해의 일출을 바라보는 것을 말하며, 옛 한송정으로서 현 강릉비행장 동쪽에 위치하였으며, 경포대의 正東을 가리킨다. 새벽에 대에 올라 동해를 바라보면, 용광로처럼 붉게 타오르는 웅장한 태양이 바다를 온통 붉게 물들이며 서서히 떠오르는 순간은 신비스러울만큼 황홀하고 아름답다. 이 장엄하고 경의로운 일출을 첫째로 꼽았다.\n \n2 죽도명월\n호수 동쪽에 있는 섬모양의 작은 산으로서 산죽이 무성하여 죽도라고 불렸는데 현 현대호텔 자리이다. 동쪽 수평선 너머에서 솟아오르는 보름달의 달빛이 죽도의 대나무 사이를 뚫어 그 빛이 호수에 비칠 때 일어나는 그림같은 장관을 죽도 명월이라 하였다. 멀리는 하늘의 달과 가까이는 바다와 호수의 달이 잇닿아 월주를 이루고 은파위의 월굴이 백해중으로부터 호심을 꿰뚫어 대 앞에까지 수십리를 뻗친, 장엄하고도  기묘한 전망을 찬미한 것이다.\n\n3 강문어화\n강문은 경포대에서 동쪽호수 하구에 있는 곳으로서 호수와 바다를 상통 교류케

In [70]:
for i in tqdm(range(2677,len(df))):
    df['overview'][i] = df['overview'][i].replace('休靜','').replace('黃嶺庵記','') \
    .replace('[','').replace(')','').replace('(','').replace(']','').replace('*','').replace('m','').replace('-','').replace('~','')\
    .replace('k','').replace(',','').replace('.','').replace('\n','')

  0%|                                                                                        | 0/14309 [00:00<?, ?it/s]C:\Users\gram\AppData\Local\Temp\ipykernel_20316\4023705023.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overview'][i] = df['overview'][i].replace('休靜','').replace('黃嶺庵記','') \
100%|████████████████████████████████████████████████████████████████████████████| 14309/14309 [05:43<00:00, 41.63it/s]


In [71]:
komoran = Komoran()

tagged_corpus_list_tr = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    text = row['overview']
    #tag = row['id']
    tagged_corpus_list_tr.append(TaggedDocument(tags=['tweet_'+str(index)], words=komoran.morphs(text)))

print('문서의 수 :', len(tagged_corpus_list_tr))

100%|███████████████████████████████████████████████████████████████████████████| 16986/16986 [02:13<00:00, 127.14it/s]

문서의 수 : 16986


In [72]:
model_d2v = gensim.models.Doc2Vec(dm=1, # dm = 1 for ‘distributed memory’ model
                                  dm_mean=1, # dm_mean = 1 for using mean of the context word vectors
                                  vector_size=250, # no. of desired features
                                  window=5, # width of the context window                                  
                                  negative=7, # if > 0 then negative sampling will be used
                                  min_count=3, # Ignores all words with total frequency lower than 5.                                  
                                  workers=32, # no. of cores                                  
                                  alpha=0.1, # learning rate                                  
                                  seed = 1992, # for reproducibility
                                 ) 

In [74]:
model_d2v.build_vocab([i for i in tqdm(tagged_corpus_list_tr)])

model_d2v.train(tagged_corpus_list_tr, total_examples= len(df), epochs=100)

100%|███████████████████████████████████████████████████████████████████████| 16986/16986 [00:00<00:00, 2413429.80it/s]


In [75]:
docvec_arrays = np.zeros((len(df), 250)) 
for i in range(0,len(df)):
    docvec_arrays[i,:] = model_d2v.docvecs[i].reshape((1,250))
doc_vectors = pd.DataFrame(docvec_arrays) 
doc_vectors.shape

C:\Users\gram\AppData\Local\Temp\ipykernel_20316\1940117879.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  docvec_arrays[i,:] = model_d2v.docvecs[i].reshape((1,250))


(16986, 250)

In [76]:
doc_vectors.to_csv('text_vectors.csv')

# Image Vector Extraction

In [77]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':30,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':256,
    'SEED':41
}
# 이미지 사이즈, 에폭, 학습률, 배치사이즈, 시드 고정

In [78]:
cnn_extract = timm.create_model('seresnext26d_32x4d', pretrained=True)
cnn_extract=nn.Sequential(*list(cnn_extract.children())[:-1],
                              nn.Linear(2048,1024),
                              nn.ReLU(),
                              nn.Linear(1024,250),
                              nn.ReLU())

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/seresnext26d_32x4d-80fa48a3.pth" to C:\Users\gram/.cache\torch\hub\checkpoints\seresnext26d_32x4d-80fa48a3.pth


In [79]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [80]:
imag_fetures=[]
img_path_list= df['img_path']
for ind in tqdm(range(0,len(df))):
    image = PIL.Image.open(df['img_path'][ind]).convert('RGB')
    img = transform(image)
    img = img.reshape(1,3,224,224)
    img_feature = cnn_extract(img)
    img_feature = img_feature[0].detach().numpy()
    imag_fetures.append(img_feature)

100%|████████████████████████████████████████████████████████████████████████████| 16986/16986 [55:35<00:00,  5.09it/s]


In [81]:
img_vec = pd.DataFrame(imag_fetures)
img_vec.to_csv('./img_vecs.csv')